In [285]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

##### Problemi:
Da li vozač osvaja bodove tokom trke ili ne?<br>
Da li će vozač u narednom krugu da izgubi, osvoji ili ostane na istoj poziciji?<br>
Određivanje vremena narednog kruga.

# Loading data

In [286]:
circuits = pd.read_csv('data/circuits.csv')
constructor_results = pd.read_csv('data/constructor_results.csv')
constructor_standings = pd.read_csv('data/constructor_standings.csv')
constructors = pd.read_csv('data/constructors.csv')
driver_standings = pd.read_csv('data/driver_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
lap_times = pd.read_csv('data/lap_times.csv')
pit_stops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')
races = pd.read_csv('data/races.csv')
results = pd.read_csv('data/results.csv')
seasons = pd.read_csv('data/seasons.csv')
sprint_results = pd.read_csv('data/sprint_results.csv')
status = pd.read_csv('data/status.csv')

# Understanding, refining and cleaning data

In [287]:
driver_standings.tail(10)
#drivers[drivers['driverId'] == 830]

,driverStandingsId,raceId,driverId,points,position,positionText,wins
34853,73261,1144,848,12.0,16,16,0
34854,73262,1144,807,41.0,11,11,0
34855,73263,1144,839,23.0,14,14,0
34856,73264,1144,842,42.0,10,10,0
34857,73265,1144,822,0.0,22,22,0
34858,73266,1144,858,0.0,23,23,0
34859,73267,1144,860,7.0,18,18,0
34860,73268,1144,861,5.0,19,19,0
34861,73269,1144,859,4.0,21,21,0
34862,73270,1144,862,0.0,24,24,0


In [288]:
races = races[races['year'] >= 2020]

In [289]:
races.head(10)

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1018,1031,2020,1,70,Austrian Grand Prix,2020-07-05,13:10:00,http://en.wikipedia.org/wiki/2020_Austrian_Gra...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1019,1032,2020,2,70,Styrian Grand Prix,2020-07-12,13:10:00,http://en.wikipedia.org/wiki/2020_Styrian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1020,1033,2020,3,11,Hungarian Grand Prix,2020-07-19,13:10:00,http://en.wikipedia.org/wiki/2020_Hungarian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1021,1034,2020,4,9,British Grand Prix,2020-08-02,13:10:00,http://en.wikipedia.org/wiki/2020_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1022,1035,2020,5,9,70th Anniversary Grand Prix,2020-08-09,13:10:00,http://en.wikipedia.org/wiki/70th_Anniversary_...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1023,1036,2020,6,4,Spanish Grand Prix,2020-08-16,13:10:00,http://en.wikipedia.org/wiki/2020_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1024,1037,2020,7,13,Belgian Grand Prix,2020-08-30,13:10:00,http://en.wikipedia.org/wiki/2020_Belgian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1025,1038,2020,8,14,Italian Grand Prix,2020-09-06,13:10:00,http://en.wikipedia.org/wiki/2020_Italian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1026,1039,2020,9,76,Tuscan Grand Prix,2020-09-13,13:10:00,http://en.wikipedia.org/wiki/2020_Tuscan_Grand...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1027,1040,2020,10,71,Russian Grand Prix,2020-09-27,11:10:00,http://en.wikipedia.org/wiki/2020_Russian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [290]:
cols = ['raceId', 'circuitId', 'year', 'round', 'name', 'date', 'time']
df = races[cols]
print(df.shape)

(107, 7)


1031 je prva trka u 2020. godini

Spajamo left, što znači da uzimamo samo iz df skupa raceId, i spajamo sa raceId iz quali

In [291]:
df = df.merge(qualifying, on='raceId', how='left')

In [292]:
print(df.shape)
df['grid'] = df['position']
df = df.drop(columns='position')
df.head()

(2138, 15)


,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5


Ovde cemo da uzmemo deo constructor_standings i driver_standings skupova gde su podaci noviji od 12.2019. godine. Posto red u ovim kolonama predstavlja situaciju nakon trke, mi ćemo da kreiramo nove kolone koje će za posmatranu trku (raceId) da gledaju prethodne rezultate - tj. nakon prethodne trke - tj. pre posmatrane.

In [293]:
races.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
1018,1031,2020,1,70,Austrian Grand Prix,2020-07-05,13:10:00,http://en.wikipedia.org/wiki/2020_Austrian_Gra...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1019,1032,2020,2,70,Styrian Grand Prix,2020-07-12,13:10:00,http://en.wikipedia.org/wiki/2020_Styrian_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1020,1033,2020,3,11,Hungarian Grand Prix,2020-07-19,13:10:00,http://en.wikipedia.org/wiki/2020_Hungarian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1021,1034,2020,4,9,British Grand Prix,2020-08-02,13:10:00,http://en.wikipedia.org/wiki/2020_British_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1022,1035,2020,5,9,70th Anniversary Grand Prix,2020-08-09,13:10:00,http://en.wikipedia.org/wiki/70th_Anniversary_...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [294]:
driver_standings = driver_standings[driver_standings['raceId'] >= 1031]
constructor_standings = constructor_standings[constructor_standings['raceId'] >= 1031]

In [295]:
print('Constructor standings shape: ', constructor_standings.shape)
print('Driver standings shape: ', driver_standings.shape)
driver_standings.head()

Constructor standings shape:  (1070, 7)
Driver standings shape:  (2256, 7)


,driverStandingsId,raceId,driverId,points,position,positionText,wins
32566,69888,1034,830,52.0,3,3,0
32567,69887,1034,817,20.0,9,9,0
32568,69886,1034,840,20.0,8,8,0
32569,69885,1034,825,1.0,16,16,0
32570,69884,1034,154,0.0,20,20,0


In [296]:
driver_standings = driver_standings.sort_values(by=['driverId', 'raceId'])
constructor_standings = constructor_standings.sort_values(by=['constructorId', 'raceId'])

In [297]:
driver_standings['before_race_points'] = driver_standings.groupby('driverId')['points'].shift(1)
driver_standings['before_race_position'] = driver_standings.groupby('driverId')['position'].shift(1)
driver_standings['before_race_wins'] = driver_standings.groupby('driverId')['wins'].shift(1)

driver_standings.loc[driver_standings['raceId'] == 1031, ['before_race_points',	'before_race_position',	'before_race_wins']] = [0, 0, 0]

driver_standings.head(10)

,driverStandingsId,raceId,driverId,points,position,positionText,wins,before_race_points,before_race_position,before_race_wins
32604,69998,1031,1,12.0,4,4,0,0.0,0.0,0.0
32611,69812,1032,1,37.0,2,2,1,12.0,4.0,0.0
32637,69852,1033,1,63.0,1,1,2,37.0,2.0,1.0
32582,69872,1034,1,88.0,1,1,3,63.0,1.0,2.0
32652,69893,1035,1,107.0,1,1,3,88.0,1.0,3.0
32673,69914,1036,1,132.0,1,1,4,107.0,1.0,3.0
32694,69935,1037,1,157.0,1,1,5,132.0,1.0,4.0
32715,69956,1038,1,164.0,1,1,5,157.0,1.0,5.0
32736,69977,1039,1,190.0,1,1,6,164.0,1.0,5.0
32755,70018,1040,1,205.0,1,1,6,190.0,1.0,6.0


In [298]:
constructor_standings['before_race_ctor_points'] = constructor_standings.groupby('constructorId')['points'].shift(1)
constructor_standings['before_race_ctor_position'] = constructor_standings.groupby('constructorId')['position'].shift(1)
constructor_standings['before_race_ctor_wins'] = constructor_standings.groupby('constructorId')['wins'].shift(1)

constructor_standings.loc[constructor_standings['raceId'] == 1031, ['before_race_ctor_points', 'before_race_ctor_position', 'before_race_ctor_wins']] = [0, 0, 0]

constructor_standings.head(10)

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins
12334,27575,1031,1,26.0,2,2,0,0.0,0.0,0.0
12339,27485,1032,1,39.0,2,2,0,26.0,2.0,0.0
12353,27505,1033,1,41.0,3,3,0,39.0,2.0,0.0
12323,27515,1034,1,51.0,3,3,0,41.0,3.0,0.0
12359,27525,1035,1,53.0,4,4,0,51.0,3.0,0.0
12369,27535,1036,1,62.0,4,4,0,53.0,4.0,0.0
12379,27545,1037,1,68.0,3,3,0,62.0,4.0,0.0
12389,27555,1038,1,98.0,3,3,0,68.0,3.0,0.0
12399,27565,1039,1,106.0,3,3,0,98.0,3.0,0.0
12408,27585,1040,1,106.0,3,3,0,106.0,3.0,0.0


In [299]:
driver_features = driver_standings[['raceId', 'driverId', 'before_race_points', 'before_race_position', 'before_race_wins']]
constructor_features = constructor_standings[['raceId', 'constructorId', 'before_race_ctor_points', 'before_race_ctor_position', 
                                                                                                    'before_race_ctor_wins']]

In [300]:
df = df.merge(driver_features, on=['raceId', 'driverId'], how='left')
df = df.merge(constructor_features, on=['raceId', 'constructorId'], how='left')

In [301]:
print(df.shape)
df.head(5)

(2138, 21)


,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1,0.0,0.0,0.0,0.0,0.0,0.0
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2,0.0,0.0,0.0,0.0,0.0,0.0
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3,0.0,0.0,0.0,0.0,0.0,0.0
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4,0.0,0.0,0.0,0.0,0.0,0.0
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5,0.0,0.0,0.0,0.0,0.0,0.0


Spajamo df sa drivers da bismo dobili datum rođenja, da bismo napravili feature broj godina.

In [302]:
drivers_red = drivers[['driverId', 'dob']]

In [303]:
df = df.merge(drivers_red, on='driverId', how='left')

In [304]:
df.head()

,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1,0.0,0.0,0.0,0.0,0.0,0.0,1989-08-28
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2,0.0,0.0,0.0,0.0,0.0,0.0,1985-01-07
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3,0.0,0.0,0.0,0.0,0.0,0.0,1997-09-30
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4,0.0,0.0,0.0,0.0,0.0,0.0,1999-11-13
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5,0.0,0.0,0.0,0.0,0.0,0.0,1996-03-23


Za name - nazive staza ne postoje null vrednosti tj. na ili nešto slično. Pomoću metode unique() sam proverio i dobijam isključivo nazive staza.

In [305]:
df['name'].unique()

array(['Austrian Grand Prix', 'Styrian Grand Prix',
       'Hungarian Grand Prix', 'British Grand Prix',
       '70th Anniversary Grand Prix', 'Spanish Grand Prix',
       'Belgian Grand Prix', 'Italian Grand Prix', 'Tuscan Grand Prix',
       'Russian Grand Prix', 'Eifel Grand Prix', 'Portuguese Grand Prix',
       'Emilia Romagna Grand Prix', 'Turkish Grand Prix',
       'Bahrain Grand Prix', 'Sakhir Grand Prix', 'Abu Dhabi Grand Prix',
       'Qatar Grand Prix', 'Monaco Grand Prix', 'Azerbaijan Grand Prix',
       'French Grand Prix', 'Dutch Grand Prix',
       'United States Grand Prix', 'Mexico City Grand Prix',
       'São Paulo Grand Prix', 'Saudi Arabian Grand Prix',
       'Australian Grand Prix', 'Miami Grand Prix', 'Canadian Grand Prix',
       'Singapore Grand Prix', 'Japanese Grand Prix',
       'Las Vegas Grand Prix', 'Chinese Grand Prix'], dtype=object)

In [306]:
df['date'] = pd.to_datetime(df['date'])
df['dob'] = pd.to_datetime(df['dob'])
print(df.dtypes)
df.head()

raceId                                int64
circuitId                             int64
year                                  int64
round                                 int64
name                                 object
date                         datetime64[ns]
time                                 object
qualifyId                             int64
driverId                              int64
constructorId                         int64
number                                int64
q1                                   object
q2                                   object
q3                                   object
grid                                  int64
before_race_points                  float64
before_race_position                float64
before_race_wins                    float64
before_race_ctor_points             float64
before_race_ctor_position           float64
before_race_ctor_wins               float64
dob                          datetime64[ns]
dtype: object


,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1,0.0,0.0,0.0,0.0,0.0,0.0,1989-08-28
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2,0.0,0.0,0.0,0.0,0.0,0.0,1985-01-07
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3,0.0,0.0,0.0,0.0,0.0,0.0,1997-09-30
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4,0.0,0.0,0.0,0.0,0.0,0.0,1999-11-13
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5,0.0,0.0,0.0,0.0,0.0,0.0,1996-03-23


In [307]:
def time_to_miliseconds(t):
    if pd.isna(t):
        return None
    try:
        parts = t.split(':')
        minutes = int(parts[0])
        seconds = float(parts[1])
        total_ms = (minutes * 60 + seconds) * 1000
        return int(total_ms)
    except:
        return None

In [308]:
df['race_time_ms'] = df['time'].apply(time_to_miliseconds)
df['q1_ms'] = df['q1'].apply(time_to_miliseconds)
df['q2_ms'] = df['q2'].apply(time_to_miliseconds)
df['q3_ms'] = df['q3'].apply(time_to_miliseconds)

In [309]:
df['driver_age'] = (df['date'] - df['dob']).dt.days / 365.25

In [310]:
df = df.replace("\\N", np.nan)
print(df.isna().sum())

raceId                          0
circuitId                       0
year                            0
round                           0
name                            0
date                            0
time                            0
qualifyId                       0
driverId                        0
constructorId                   0
number                          0
q1                             21
q2                            551
q3                           1093
grid                            0
before_race_points             15
before_race_position           15
before_race_wins               15
before_race_ctor_points         8
before_race_ctor_position       8
before_race_ctor_wins           8
dob                             0
race_time_ms                    0
q1_ms                          21
q2_ms                         551
q3_ms                        1093
driver_age                      0
dtype: int64


In [311]:
df.head()

,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob,race_time_ms,q1_ms,q2_ms,q3_ms,driver_age
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1,0.0,0.0,0.0,0.0,0.0,0.0,1989-08-28,790000,64111.0,63015.0,62939.0,30.852841
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2,0.0,0.0,0.0,0.0,0.0,0.0,1985-01-07,790000,64198.0,63096.0,62951.0,35.490760
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3,0.0,0.0,0.0,0.0,0.0,0.0,1997-09-30,790000,64024.0,64000.0,63477.0,22.762491
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4,0.0,0.0,0.0,0.0,0.0,0.0,1999-11-13,790000,64605.0,63819.0,63626.0,20.643395
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5,0.0,0.0,0.0,0.0,0.0,0.0,1996-03-23,790000,64661.0,63746.0,63868.0,24.284736


In [312]:
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')

Kreiramo feature doba dana trke - ujutru, popodne, uvece. To radimo tako što ćemo iz time kolone izvesti hour.

In [313]:
def part_of_day(h):
    if 6 <= h < 12:
        return "morning"
    elif 12 <= h < 18:
        return "afternoon"
    else:
        return "evening"

In [314]:
df['hour'] = df['time'].dt.hour
df['race_part_of_day'] = df['hour'].apply(part_of_day)

Ovde ću da kreiram novi feature koji označava u kom q delu je vozač ispao. 1-q1, 2-q2, 3-q3.

In [315]:
def get_round(row):
    if pd.isna(row['q2']):
        return 1 # ispao u q1
    elif pd.isna(row['q3']):
        return 2 # ispao u q2
    else:
        return 3 # dosao u q3

In [316]:
df['qualifying_round'] = df.apply(get_round ,axis=1)

In [317]:
df.head()

,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob,race_time_ms,q1_ms,q2_ms,q3_ms,driver_age,hour,race_part_of_day,qualifying_round
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1,0.0,0.0,0.0,0.0,0.0,0.0,1989-08-28,790000,64111.0,63015.0,62939.0,30.852841,13,afternoon,3
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2,0.0,0.0,0.0,0.0,0.0,0.0,1985-01-07,790000,64198.0,63096.0,62951.0,35.490760,13,afternoon,3
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3,0.0,0.0,0.0,0.0,0.0,0.0,1997-09-30,790000,64024.0,64000.0,63477.0,22.762491,13,afternoon,3
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4,0.0,0.0,0.0,0.0,0.0,0.0,1999-11-13,790000,64605.0,63819.0,63626.0,20.643395,13,afternoon,3
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5,0.0,0.0,0.0,0.0,0.0,0.0,1996-03-23,790000,64661.0,63746.0,63868.0,24.284736,13,afternoon,3


In [318]:
results = results[results['raceId'] >= 1031]

In [319]:
results = results.replace("\\N", np.nan)
results.isna().sum()

resultId             0
raceId               0
driverId             0
constructorId        0
number               0
grid                 0
position           271
positionText         0
positionOrder        0
points               0
laps                 0
time               860
milliseconds       860
fastestLap         109
rank                 0
fastestLapTime     109
fastestLapSpeed    109
statusId             0
dtype: int64

In [320]:
results = results[['raceId', 'driverId', 'constructorId', 'positionText', 'rank']]
df = df.merge(results, on=['raceId', 'driverId', 'constructorId'], how='left')
df.shape

(2138, 32)

Ovde kreiramo kolonu najbrži krug bonus, koja označava koji vozač je uzeo najbrži krug. - DATA LEAKAGE?

In [321]:
# df['fastest_lap_bonus'] = df['rank'].apply(lambda x: 1 if x == 1 else 0)

Ovde kreiramo ciljanu promenljivu

In [322]:
print(df['positionText'].unique())

df['final_position_num'] = pd.to_numeric(df['positionText'], errors='coerce')
print(df['final_position_num'].unique())

['1' '4' 'R' '3' '13' '6' '2' '5' '10' '7' '12' '8' '9' '11' '15' '16'
 '17' '14' '18' '19' 'W' '20' 'D']
[ 1.  4. nan  3. 13.  6.  2.  5. 10.  7. 12.  8.  9. 11. 15. 16. 17. 14.
 18. 19. 20.]


In [323]:
df['scored_points'] = df['final_position_num'].apply(lambda x: 1 if x <= 10 else 0)
df['scored_points'].unique()

array([1, 0], dtype=int64)

In [324]:
df.head()

,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob,race_time_ms,q1_ms,q2_ms,q3_ms,driver_age,hour,race_part_of_day,qualifying_round,positionText,rank,final_position_num,scored_points
0,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8378,822,131,77,1:04.111,1:03.015,1:02.939,1,0.0,0.0,0.0,0.0,0.0,0.0,1989-08-28,790000,64111.0,63015.0,62939.0,30.852841,13,afternoon,3,1,2,1.0,1
1,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8379,1,131,44,1:04.198,1:03.096,1:02.951,2,0.0,0.0,0.0,0.0,0.0,0.0,1985-01-07,790000,64198.0,63096.0,62951.0,35.490760,13,afternoon,3,4,3,4.0,1
2,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8380,830,9,33,1:04.024,1:04.000,1:03.477,3,0.0,0.0,0.0,0.0,0.0,0.0,1997-09-30,790000,64024.0,64000.0,63477.0,22.762491,13,afternoon,3,R,15,NaN,0
3,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8381,846,1,4,1:04.606,1:03.819,1:03.626,4,0.0,0.0,0.0,0.0,0.0,0.0,1999-11-13,790000,64605.0,63819.0,63626.0,20.643395,13,afternoon,3,3,1,3.0,1
4,1031,70,2020,1,Austrian Grand Prix,2020-07-05,1900-01-01 13:10:00,8382,848,9,23,1:04.661,1:03.746,1:03.868,5,0.0,0.0,0.0,0.0,0.0,0.0,1996-03-23,790000,64661.0,63746.0,63868.0,24.284736,13,afternoon,3,13,7,13.0,0


<b>year</b> - godina trke <br>
<b>round</b> - koja je runda u pitanju - trka <br>
<b>date</b> - datum trke <br>
<b>time</b> - vreme trke <br>
<b>number</b> - broj vozača <br>
<b>grid</b> - pozicija sa koje vozač startuje <br>
<b>before_race_(points, position, wins)</b> - koliko posmatrani vozač ima poena, na kojoj se poziciji nalazi (na nivou šampionata - godine) i koliko ima pobeda do sada - tj. do tekuće trke <br>
<b>before_race_ctor(points, position, wins)</b> - isto kao i za vozače, samo za konstruktore <br>
<b>name_(naziv staze)</b> - ohe staze<br>
<b>q1 q2 q3</b> - vremena za q1, q2 i q3 <br>
<b>race_time_ms</b> - vreme trke u milisekundama
<b>hour</b> - u koliko h počinje trka <br>
<b>race_part_of_day</b> - oh morning, afternoon, evening<br>
<b>qulifying_round</b> - u kom delu je vozač ispao 3 - došao do q3, 2 - došao do q2 i ispao, 1 - ispao u q1 <br>
<b>positionText</b> - osvojeno mesto ima i R, W i još nešto <b> - ZA BRISANJE?</b><br>
<b>rank</b> - koji vozač je uzeo najbrži krug - 1 najbolje vreme -> 20<b> - ZA BRISANJE?</b><br>
<b>fastest_lap_bonus</b> - ko je uzeo najbrži krug [0,1]<b> - ZA BRISANJE?</b><br>
<b>final_position_num</b> - krajnjna mesta - ima NaN - nije završio<b> - ZA BRISANJE?</b> ili da se ostavi kao feature, za prethodni krug<br>
<b>scored_points</b> - ciljana promenljiva - vozač ostvario bodovi ili nije [0,1]<b> - y</b><br>


In [325]:
df.isna().sum()

raceId                          0
circuitId                       0
year                            0
round                           0
name                            0
date                            0
time                            0
qualifyId                       0
driverId                        0
constructorId                   0
number                          0
q1                             21
q2                            551
q3                           1093
grid                            0
before_race_points             15
before_race_position           15
before_race_wins               15
before_race_ctor_points         8
before_race_ctor_position       8
before_race_ctor_wins           8
dob                             0
race_time_ms                    0
q1_ms                          21
q2_ms                         551
q3_ms                        1093
driver_age                      0
hour                            0
race_part_of_day                0
qualifying_rou

In [326]:
df.describe()

,raceId,circuitId,year,round,date,time,qualifyId,driverId,constructorId,number,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob,race_time_ms,q1_ms,q2_ms,q3_ms,driver_age,hour,qualifying_round,final_position_num,scored_points
count,2138.000000,2138.000000,2138.000000,2138.000000,2138,2138,2138.000000,2138.000000,2138.000000,2138.000000,2138.000000,2123.000000,2123.000000,2123.000000,2130.000000,2130.000000,2130.000000,2138,2.138000e+03,2117.000000,1587.000000,1045.000000,2138.000000,2138.000000,2138.000000,1867.000000,2138.000000
mean,1088.503742,32.559869,2022.130496,11.333957,2022-09-22 20:24:28.288119808,1900-01-01 13:34:40.636108544,9473.799813,719.040692,92.881665,27.799345,10.491113,58.942770,10.526613,0.554404,117.778873,5.441784,1.114554,1994-04-03 17:03:05.219831552,8.146773e+05,85496.521020,84519.494014,84150.616268,28.471293,13.537418,2.231057,9.311730,0.500468
min,1031.000000,1.000000,2020.000000,1.000000,2020-07-05 00:00:00,1900-01-01 04:00:00,8378.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1979-10-17 00:00:00,2.400000e+05,53904.000000,53647.000000,53377.000000,18.836413,4.000000,1.000000,1.000000,0.000000
25%,1060.000000,9.000000,2021.000000,6.000000,2021-07-18 00:00:00,1900-01-01 13:00:00,8929.250000,817.000000,6.000000,10.000000,5.250000,3.500000,5.000000,0.000000,8.000000,3.000000,0.000000,1989-08-28 00:00:00,7.800000e+05,77189.000000,76543.500000,76170.000000,24.061602,13.000000,1.000000,5.000000,0.000000
50%,1088.000000,21.000000,2022.000000,11.000000,2022-09-04 00:00:00,1900-01-01 13:00:00,9482.500000,839.000000,51.000000,20.000000,10.000000,22.000000,10.000000,0.000000,46.000000,5.000000,0.000000,1996-02-07 00:00:00,7.800000e+05,84061.000000,83420.000000,83557.000000,26.663929,13.000000,2.000000,9.000000,1.000000
75%,1118.000000,70.000000,2023.000000,17.000000,2023-11-05 00:00:00,1900-01-01 15:00:00,10016.750000,847.000000,210.000000,44.000000,15.000000,83.000000,16.000000,0.000000,163.000000,8.000000,0.000000,1998-10-29 00:00:00,9.000000e+05,92098.000000,90808.500000,90554.000000,32.431896,15.000000,3.000000,14.000000,1.000000
max,1144.000000,80.000000,2024.000000,24.000000,2024-12-08 00:00:00,1900-01-01 20:00:00,10551.000000,862.000000,215.000000,99.000000,20.000000,575.000000,23.000000,19.000000,860.000000,10.000000,21.000000,2005-05-08 00:00:00,1.200000e+06,141611.000000,126688.000000,123513.000000,43.362081,20.000000,3.000000,20.000000,1.000000
std,33.241142,28.463975,1.388264,6.370320,NaN,NaN,627.704242,290.465754,89.355219,24.224605,5.762997,82.631161,6.013700,1.833615,158.029901,2.909572,2.878146,NaN,2.073265e+05,12844.737961,12230.518169,12062.071091,5.603073,3.439531,0.832724,5.171785,0.500117


In [327]:
# !pip install xgboost
# !pip install lightgbm - nije instaliran

In [328]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [329]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)

In [330]:
df.dtypes

raceId                                int64
circuitId                             int64
year                                  int64
round                                 int64
name                                 object
date                         datetime64[ns]
time                         datetime64[ns]
qualifyId                             int64
driverId                              int64
constructorId                         int64
number                                int64
q1                                   object
q2                                   object
q3                                   object
grid                                  int64
before_race_points                  float64
before_race_position                float64
before_race_wins                    float64
before_race_ctor_points             float64
before_race_ctor_position           float64
before_race_ctor_wins               float64
dob                          datetime64[ns]
race_time_ms                    

# Helper functions

In [331]:
def evaluate_classifier(model_name, y_true, y_pred, y_proba=None):
    print('==========', model_name, '==========')

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    print(f'Accuracy : {acc:.4f}')
    print(f'Precision: {prec:.4f}')
    print(f'Recall   : {rec:.4f}')
    print(f'F1-score : {f1:.4f}')

    if y_proba is not None:
        roc = roc_auc_score(y_true, y_proba)
        print(f'ROC-AUC  : {roc:.4f}')

    print('Confusion matrix:')
    print(confusion_matrix(y_true, y_pred), '\n')

# First model training and testing

#### Time based split - HOLD-OUT

In [332]:
train_df = df[df['year'] <= 2023]
test_df  = df[df['year'] == 2024]

In [333]:
y = df['scored_points']
y.mean()

0.500467726847521

In [334]:
test_df.head()

,raceId,circuitId,year,round,name,date,time,qualifyId,driverId,constructorId,number,q1,q2,q3,grid,before_race_points,before_race_position,before_race_wins,before_race_ctor_points,before_race_ctor_position,before_race_ctor_wins,dob,race_time_ms,q1_ms,q2_ms,q3_ms,driver_age,hour,race_part_of_day,qualifying_round,positionText,rank,final_position_num,scored_points
1659,1121,3,2024,1,Bahrain Grand Prix,2024-03-02,1900-01-01 15:00:00,10073,830,9,1,1:30.031,1:29.374,1:29.179,1,575.0,1.0,19.0,860.0,1.0,21.0,1997-09-30,900000,90031.0,89374.0,89179.0,26.420260,15,afternoon,3,1,1,1.0,1
1660,1121,3,2024,1,Bahrain Grand Prix,2024-03-02,1900-01-01 15:00:00,10074,844,6,16,1:30.243,1:29.165,1:29.407,2,206.0,5.0,0.0,406.0,3.0,1.0,1997-10-16,900000,90243.0,89164.0,89407.0,26.376454,15,afternoon,3,4,2,4.0,1
1661,1121,3,2024,1,Bahrain Grand Prix,2024-03-02,1900-01-01 15:00:00,10075,847,131,63,1:30.350,1:29.922,1:29.485,3,175.0,8.0,0.0,409.0,2.0,0.0,1998-02-15,900000,90350.0,89922.0,89485.0,26.042437,15,afternoon,3,5,12,5.0,1
1662,1121,3,2024,1,Bahrain Grand Prix,2024-03-02,1900-01-01 15:00:00,10076,832,6,55,1:29.909,1:29.573,1:29.507,4,200.0,7.0,1.0,406.0,3.0,1.0,1994-09-01,900000,89908.0,89573.0,89507.0,29.500342,15,afternoon,3,3,6,3.0,1
1663,1121,3,2024,1,Bahrain Grand Prix,2024-03-02,1900-01-01 15:00:00,10077,815,9,11,1:30.221,1:29.932,1:29.537,5,285.0,2.0,2.0,860.0,1.0,21.0,1990-01-26,900000,90221.0,89932.0,89537.0,34.097194,15,afternoon,3,2,4,2.0,1


In [335]:
X_train = train_df.drop(columns=['raceId', 'circuitId', 'year', 'date', 'time', 'qualifyId', 'driverId', 'constructorId',
                     'number', 'q1', 'q2', 'q3', 'dob', 'race_time_ms', 'q1_ms', 'q2_ms', 'q3_ms', 'hour',
                     'positionText', 'rank', 'final_position_num', 'scored_points', 'qualifying_round', 'grid'])
y_train = train_df['scored_points']

X_test = test_df.drop(columns=['raceId', 'circuitId', 'year', 'date', 'time', 'qualifyId', 'driverId', 'constructorId',
                     'number', 'q1', 'q2', 'q3', 'dob', 'race_time_ms', 'q1_ms', 'q2_ms', 'q3_ms', 'hour',
                     'positionText', 'rank', 'final_position_num', 'scored_points', 'qualifying_round', 'grid'])
y_test = test_df['scored_points']

#### Pipelines

Needs to be updated every time a new feature is added.

In [336]:
categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=['number']).columns.tolist()

In [337]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

In [338]:
pipeline_hd_log_reg = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", LogisticRegression())
])

pipeline_hd_rf =Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", RandomForestClassifier())
])

pipeline_hd_xgb =Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", XGBClassifier())
])

In [339]:
pipelines = {
    "Logistic Regression": pipeline_hd_log_reg,
    "Random Forest": pipeline_hd_rf,
    "XGBoost": pipeline_hd_xgb
}

#### First model evaluation

In [340]:
for name, pipeline in pipelines.items():
    print(f"\nTraining {name} ...")

    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)

    y_proba = pipeline.predict_proba(X_test)[:, 1]

    evaluate_classifier(
        model_name=name,
        y_true=y_test,
        y_pred=y_pred,
        y_proba=y_proba
    )


Training Logistic Regression ...
========== Logistic Regression ==========
Accuracy : 0.8058
Precision: 0.8267
Recall   : 0.7750
F1-score : 0.8000
ROC-AUC  : 0.8591
Confusion matrix:
[[200  39]
 [ 54 186]] 


Training Random Forest ...
========== Random Forest ==========
Accuracy : 0.7787
Precision: 0.7746
Recall   : 0.7875
F1-score : 0.7810
ROC-AUC  : 0.8395
Confusion matrix:
[[184  55]
 [ 51 189]] 


Training XGBoost ...
========== XGBoost ==========
Accuracy : 0.7140
Precision: 0.6958
Recall   : 0.7625
F1-score : 0.7276
ROC-AUC  : 0.7919
Confusion matrix:
[[159  80]
 [ 57 183]] 



#### Time-based split - Expanding window

In [341]:
y = df['scored_points']
cols_to_drop = ['raceId', 'circuitId', 'year', 'date', 'time', 'qualifyId', 'driverId', 'constructorId',
                     'number', 'q1', 'q2', 'q3', 'dob', 'race_time_ms', 'q1_ms', 'q2_ms', 'q3_ms', 'hour',
                     'positionText', 'rank', 'final_position_num', 'scored_points', 'qualifying_round', 'grid']
features = df.drop(columns=(cols_to_drop))

In [342]:
from sklearn.base import clone

years = sorted(df['year']).unique()

results = []

for i in range(1, len(years)):
    train_y = years[i:]
    test_y = years[i]

    train_df = df[df['year'].isin(train_y)]
    test_df = df[df['year'] == test_y]

    X_train = train_df[features]
    y_train = train_df[y]

    X_test = test_df[features]
    y_test = test_df[y]

    models = clone(pipelines)

    for name, model in models.items():
        print(f"\nTraining {name} ...")
    
        model.fit(X_train, y_train)
    
        y_pred = model.predict(X_test)
    
        y_proba = model.predict_proba(X_test)[:, 1]
    
        evaluate_classifier(
            model_name=name,
            y_true=y_test,
            y_pred=y_pred,
            y_proba=y_proba
        )
        

AttributeError: 'list' object has no attribute 'unique'

# Second model upgrade